In [99]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [100]:
data = pd.read_csv(r"D:\Data Science education data\Unit 1 final task\movie_bd_v5.csv")
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
932,tt0299658,45000000,306776732,Chicago,Catherine Zeta-Jones|RenÃ©e Zellweger|Queen La...,Rob Marshall,"If You Can't Be Famous, Be Infamous.",Murderesses Velma Kelly (a chanteuse and tease...,113,Action|Comedy|Crime|Drama|Music,Miramax Films|Producers Circle|Storyline Enter...,12/26/2002,6.5,2002
809,tt1305591,150000000,38992758,Mars Needs Moms,Seth Green|Joan Cusack|Dan Fogler|Breckin Meye...,Simon Wells,Mom needs a little space.,"When Martians suddenly abduct his mom, mischie...",88,Adventure|Animation|Family,Walt Disney Animation Studios,3/9/2011,5.5,2011
1225,tt2312718,22000000,43058898,Homefront,Jason Statham|James Franco|Izabela Vidovic|Win...,Gary Fleder,How far would you go to protect your home?,Phil Broker is a former DEA agent who has gone...,100,Action|Thriller,Millennium Films|Open Road Films,11/27/2013,6.4,2013
1560,tt0304141,130000000,789804554,Harry Potter and the Prisoner of Azkaban,Daniel Radcliffe|Rupert Grint|Emma Watson|Gary...,Alfonso CuarÃ³n,Something wicked this way comes.,"Harry, Ron and Hermione return to Hogwarts for...",141,Adventure|Fantasy|Family,1492 Pictures|Warner Bros.|Heyday Films|P of A...,5/31/2004,7.4,2004
1316,tt0401792,40000000,158733820,Sin City,Jessica Alba|Devon Aoki|Alexis Bledel|Powers B...,Robert Rodriguez|Frank Miller|Quentin Tarantino,Walk down the right back alley in Sin City and...,Welcome to Sin City. This town beckons to the ...,124,Action|Thriller|Crime,Dimension Films|Troublemaker Studios|Miramax,3/31/2005,7.1,2005


In [101]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [102]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...


# 1) Добавление колонки "profit":
data['profit'] = data['revenue'] - data['budget']

# 2) Добавление функции:
def get_the_most_common_genre():
    """ Получение наиболее часто встречающегося жанра с помощью класса Counter"""
    list_of_genres = []

    for x in data.genres:
        y = x.split('|')
        list_of_genres.append(y)

    def flatten_list(initial_list):
        """ Превращает лист содержащий в себе листы среди элементов в одномерный лист"""
        return [item for sublist in initial_list for item in sublist]

    list_of_genres_flat = flatten_list(list_of_genres)
    return Counter(list_of_genres_flat).most_common(1)


# 3) Добавление функции:
def make_the_key_column(x_series):
    """ создаем колонку с ключем для df с жанрами """
    list_x = []
    for x in x_series:
       if not x.startswith("tt" ):
           list_x.append(np.nan) # вставить пустоту
       else:
           list_x.append(x)
    key_series = pd.Series(list_x)
    key_series = key_series.fillna(method='ffill') # заполняем пустоты вышестоящим непустым значением
    return key_series


# 4) Добавление колонок с месяцем и временим года:
data_month = data.release_date.str.split(pat="/", expand=True).drop([1,2], axis=1)
data_month = data_month.astype('int64', copy=False)  # смена формата с object на числовой

data['release_month'] = data_month


seasons_dict = {12: "winter",1: "winter", 2: "winter", 3: "spring", 4: "spring", 5: "spring",
                6: "summer", 7: "summer", 8: "summer", 9: "fall", 10: "fall", 11: "fall" }

data['release_season'] = data['release_month'].apply(lambda x: seasons_dict[x])


# 5) Добавление колонок с количеством символов в названии фильмов и слов в описании:
data_temp = data['original_title']
data_len_of_title = data_temp.str.len()
data_len_of_title = data_len_of_title.astype('int64', copy=False) # смена формата с object на числовой
data['original_title_len'] = data_len_of_title

data['overview_words_count'] = data['overview'].str.count(' ') + 1 # через подсчет количества пробелов
data['overview_words_count'] = data['overview_words_count'].astype('int64', copy=False) # смена формата с object на числовой

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [103]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' # +
# если ответили верно, можете добавить комментарий со значком "+"

In [104]:
# тут пишем ваш код для решения данного вопроса:
data_temp = data.sort_values(ascending=False, by="budget")
display(data_temp.head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000,5,spring,43,75


ВАРИАНТ 2

In [105]:
# можно добавлять разные варианты решения
data_temp = data.loc[data.budget == data.budget.max()]
display(data_temp)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011,641683000,5,spring,43,75


# 2. Какой из фильмов самый длительный (в минутах)?

In [106]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)' # +

In [107]:
# Решение:
data_temp = data.sort_values(ascending=False, by="runtime")
display(data_temp.head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003,-43076064,2,winter,17,48


# 3. Какой из фильмов самый короткий (в минутах)?





In [108]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' # +

In [109]:
# Решение:
data_temp = data.sort_values(ascending=True, by="runtime")
display(data_temp.head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011,-15540000,4,spring,15,52


# 4. Какова средняя длительность фильмов?


In [110]:
answers['4']='2. 110' # +

In [111]:
# Решение:
data_temp = round(data.runtime.mean())
display(data_temp)

110

# 5. Каково медианное значение длительности фильмов? 

In [112]:
answers['5']='1. 107' # +

In [113]:
# Решение:
data_temp = round(data.runtime.median())
display(data_temp)

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [114]:
answers['6']='5. Avatar (tt0499549)' # +

In [115]:
# лучше код получения столбца profit вынести в Предобработку что в начале
# Решение:
data_temp = data.loc[data.profit == data.profit.max()]
display(data_temp)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,12/10/2009,7.1,2009,2544505847,12,winter,6,28


# 7. Какой фильм самый убыточный? 

In [116]:
answers['7']='5. The Lone Ranger (tt1210819)' # +

In [117]:
# Решение:
data_temp = data.sort_values(ascending=True, by="profit")
display(data_temp.head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,7,summer,15,68


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [118]:
answers['8']='1. 1478' # +

In [119]:
# Решение:
data_temp = data.profit.loc[data.profit > 0].count()
display(data_temp)

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [120]:
answers['9']='4. The Dark Knight (tt0468569)' # +

In [121]:
# Решение:
data_temp = data.loc[data.release_year == 2008]
data_temp2 = data_temp.loc[data_temp.profit == data_temp.profit.max()]
display(data_temp2)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,7,summer,15,67


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [122]:
answers['10']='5. The Lone Ranger (tt1210819)' # +

In [123]:
# Решение:
data_temp = data.loc[(data.release_year >= 2012) & (data.release_year <= 2014)]
data_temp2 = data_temp.loc[data_temp.profit == data_temp.profit.min()]
display(data_temp2)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090,7,summer,15,68


# 11. Какого жанра фильмов больше всего?

In [124]:
answers['11']='3. Drama' # +

In [125]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# Решение:
print(get_the_most_common_genre())

[('Drama', 782)]


ВАРИАНТ 2

In [126]:
# Решение:
genre_df = data.genres.str.split(pat="|", expand=True)
genre_series = genre_df.stack() # превращаем df с 5 столбцами в одну series
genre_series.value_counts().head(1)

Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [127]:
answers['12']='1. Drama' # +

In [128]:
# Решение:

# создаем series из плоского листа с жанрами:
genre_df = data.genres.str.split(pat="|", expand=True)
genre_df = pd.concat([data['imdb_id'], genre_df], axis=1)
genre_series = genre_df.stack()

 # создание df из двух series
df_genre_with_key = pd.DataFrame(zip(make_the_key_column(genre_series),genre_series)).\
rename(columns={0: 'imdb_id', 1: 'genre'})

# убираем строки, в которых в столбце 'genre' содержится imdb_id
df_genre_pure = df_genre_with_key[-df_genre_with_key['genre'].str.match ("tt")]

#---------------------------------------------------------------------------------------
 # видоизменяем data df для подтягивания только одного столбца
data_temp = data[['imdb_id','profit']]

# подтягиваем из data_temp суммы прибыли
df_genre_pure_with_profit = pd.merge(df_genre_pure, data_temp, on='imdb_id', how='right')

 # определяем какой жанр чаще всего имеет прибыль > 0
df_genre_pure_with_profit2 = df_genre_pure_with_profit.loc[df_genre_pure_with_profit.profit > 0]
df_genre_pure_with_profit3 = df_genre_pure_with_profit2['genre'].value_counts()
df_genre_pure_with_profit3.head(1)

Drama    560
Name: genre, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [129]:
answers['13']='5. Peter Jackson' # +

In [130]:
# Решение:

pivot = data.pivot_table(values='revenue', index=['director'], aggfunc='sum')

# превращение сводной таблицы в обычный df
pivot.columns.name = None  
pivot2 = pivot.reset_index()
pivot2 = pivot2.sort_values(by='revenue', ascending=False)
pivot2.head(1)

,director,revenue
689,Peter Jackson,6490593685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [131]:
answers['14']='3. Robert Rodriguez' # +

In [132]:
# Решение:
action_data = data[data['genres'].str.contains("Action", na=False)]
directors_df = action_data.director.str.split(pat="|", expand=True)
directors_series = directors_df.stack()
directors_series.value_counts().head(1)

Robert Rodriguez    9
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [133]:
answers['15']='3. Chris Hemsworth' # +

In [134]:
# Решение:

data2012 = data.loc[data['release_year']==2012]

cast_df = data2012.cast.str.split(pat="|", expand=True)
cast_df = pd.concat([data2012['imdb_id'], cast_df], axis=1)
cast_series = cast_df.stack()

# создание df из двух series
df_cast_with_key = pd.DataFrame(zip(make_the_key_column(cast_series), cast_series)). \
rename(columns={0: 'imdb_id', 1: 'cast'})

# убираем строки, в которых в столбце 'cast' содержится imdb_id
df_cast_pure = df_cast_with_key[-df_cast_with_key['cast'].str.match("tt")]

# видоизменяем data2012 df для подтягивания только одного столбца
data_temp = data2012[['imdb_id', 'revenue', 'original_title']]

# подтягиваем из data_temp суммы кассовых сборов
df_cast_pure_with_revenue = pd.merge(df_cast_pure, data_temp, on='imdb_id', how='right')

# определяем фильмы с каким актером принесли самые высокие сборы:
pivot = df_cast_pure_with_revenue.pivot_table(values=['revenue'], index=['cast'],
aggfunc='sum')

# трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns
pivot2 = pivot2.sort_values(by='revenue', ascending=False)
pivot2.head(1)

,cast,revenue
77,Chris Hemsworth,2027450773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [135]:
answers['16']='3. Matt Damon' # +

In [136]:
#Решение:
high_budget_data = data.loc[data.budget>data.budget.mean()]
cast_df = high_budget_data.cast.str.split(pat="|", expand=True)
cast_df = pd.concat([high_budget_data['imdb_id'], cast_df], axis=1)
cast_series = cast_df.stack()

# создание df из двух series
df_cast_with_key = pd.DataFrame(zip(make_the_key_column(cast_series), cast_series)). \
    rename(columns={0: 'imdb_id', 1: 'cast'})

# убираем строки, в которых в столбце 'cast' содержится imdb_id
df_cast_pure = df_cast_with_key[-df_cast_with_key['cast'].str.match("tt")]

# видоизменяем data df для подтягивания только одного столбца
data_temp = high_budget_data[['imdb_id', 'original_title']]

# подтягиваем из data_temp наименование фильмов
df_cast_pure_with_revenue = pd.merge(df_cast_pure, data_temp, on='imdb_id', how='right')

# определяем актера сыгравшего в наибольшем кол-ве высокобюджетных фильмов:
pivot = df_cast_pure_with_revenue.pivot_table(values=['original_title'], index=['cast'],
                                              aggfunc='count')

# трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns

pivot2 = pivot2.sort_values(by='original_title', ascending=False)

display(pivot2.head(1))

,cast,original_title
965,Matt Damon,18


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [137]:
answers['17']='2. Action' # +

In [138]:
# Решение:
data_NC = data[data['cast'].str.contains("Nicolas Cage", na=False)]
genre_df = data_NC.genres.str.split(pat="|", expand=True)
genre_df = pd.concat([data_NC['imdb_id'], genre_df], axis=1)
genre_series = genre_df.stack()

# создание df из двух series
df_genre_with_key = pd.DataFrame(zip(make_the_key_column(genre_series), genre_series)). \
    rename(columns={0: 'imdb_id', 1: 'genre'})

# убираем строки, в которых в столбце 'genre' содержится imdb_id
df_genre_pure = df_genre_with_key[-df_genre_with_key['genre'].str.match("tt")]

# Определяем в фильмах какого жанра больше всего снимался Nicolas Cage
df_genre_pure = df_genre_pure['genre'].value_counts()
df_genre_pure.head(1)

Action    17
Name: genre, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [139]:
answers['18']='1. K-19: The Widowmaker (tt0267626)' # +

In [140]:
#Решение:
data['profit'] = data.revenue - data.budget
data_PP = data[data['production_companies'].str.contains("Paramount Pictures", na=False)]
data_PP = data_PP.sort_values(by='profit', ascending=True)
display(data_PP.head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034,7,summer,20,23


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [141]:
answers['19']='5. 2015' # +

In [142]:
# Решение:
pivot_df = data.pivot_table(index='release_year',values='revenue', aggfunc='sum')

# трансформация сводной таблицы в df
pivot_df.columns.name = None  # remove categories
pivot_df2 = pivot_df.reset_index()  # index to columns
pivot_df2 = pivot_df2.loc[pivot_df2.revenue == pivot_df2.revenue.max()]
print(pivot_df2)

    release_year      revenue
15          2015  25449202382


# 20. Какой самый прибыльный год для студии Warner Bros?

In [143]:
answers['20']='1. 2014' # +

In [144]:
# Решение:
dataWB = data[data['production_companies'].str.contains("Warner Bros", na=False)]

pivot_df = dataWB.pivot_table(index='release_year',values='profit', aggfunc='sum')

# трансформация сводной таблицы в df
pivot_df.columns.name = None  # remove categories
pivot_df2 = pivot_df.reset_index()  # index to columns
pivot_df2 = pivot_df2.loc[pivot_df2.profit == pivot_df2.profit.max()]
print(pivot_df2)

    release_year      profit
14          2014  2295464519


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [145]:
answers['21']='4. Сентябрь' # +

In [146]:
# Решение:
pivot = data.pivot_table(values=['original_title'], index=['release_month'], aggfunc='count')
 
# трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns
pivot2 = pivot2.sort_values(by='original_title', ascending=False)
pivot2.head(1)

,release_month,original_title
8,9,227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [147]:
answers['22']='2. 450' # +

In [148]:
# Решение:
pivot = data.pivot_table(values=['original_title'], index=['release_season'], aggfunc='count')

# трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns
pivot2 = pivot2.loc[pivot2.release_season == 'summer']
pivot2 = pivot2.sort_values(by='original_title', ascending=False)
display(pivot2)

,release_season,original_title
2,summer,450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [149]:
answers['23']='5. Peter Jackson' # +

In [150]:
# Решение:
data_winter = data.loc[data.release_season == 'winter']
pivot = data_winter.pivot_table(values=['original_title'], index=['director'], aggfunc='count')

 # трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns
pivot2 = pivot2.sort_values(by='original_title', ascending=False)
display(pivot2.head(1))

,director,original_title
247,Peter Jackson,7


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [151]:
answers['24']='5. Four By Two Productions' # +

In [152]:
# Решение:

companies_df = data.production_companies.str.split(pat="|", expand=True)
companies_df = pd.concat([data['imdb_id'], companies_df], axis=1)
companies_series = companies_df.stack()

# создание df из двух series
df_companies_with_key = pd.DataFrame(zip(make_the_key_column(companies_series), companies_series)). \
    rename(columns={0: 'imdb_id', 1: 'production_companies'})

# убираем строки, в которых в столбце 'production_companies' содержится imdb_id
df_companies_pure = df_companies_with_key[-df_companies_with_key['production_companies'].str.match("tt")]

# видоизменяем data df для подтягивания только одного столбца
data_temp = data[['imdb_id', 'original_title_len']]

# подтягиваем из data_temp кол-ва симполов в названии
df_companies_pure_with_len = pd.merge(df_companies_pure, data_temp, on='imdb_id', how='right')

# определяем у какой компании-кинопроизводителя наиболее длинное (в среднем) название фильма
pivot = df_companies_pure_with_len.pivot_table(values=['original_title_len'], index=['production_companies'],
                                               aggfunc='mean')

# трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns
pivot2 = pivot2.sort_values(by='original_title_len', ascending=False)
display(pivot2.head(1))

,production_companies,original_title_len
583,Four By Two Productions,83.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [153]:
answers['25']='3. Midnight Picture Show' # +

In [154]:
# Решение:

companies_df = data.production_companies.str.split(pat="|", expand=True)
companies_df = pd.concat([data['imdb_id'], companies_df], axis=1)
companies_series = companies_df.stack()

# создание df из двух series
df_companies_with_key = pd.DataFrame(zip(make_the_key_column(companies_series), companies_series)). \
    rename(columns={0: 'imdb_id', 1: 'production_companies'})

# убираем строки, в которых в столбце 'production_companies' содержится imdb_id
df_companies_pure = df_companies_with_key[-df_companies_with_key['production_companies'].str.match("tt")]

 # видоизменяем data df для подтягивания только одного столбца
data_temp = data[['imdb_id', 'overview_words_count']]

# подтягиваем из data_temp кол-во слов в описании
df_companies_pure_with_OW = pd.merge(df_companies_pure, data_temp, on='imdb_id', how='right')

# определяем у какой компании-кинопроизводителя наиболее длинное (в среднем) описание фильма
pivot = df_companies_pure_with_OW.pivot_table(values=['overview_words_count'],
                                                              index=['production_companies'],aggfunc='mean')

# трансформация сводной таблицы в df
pivot.columns.name = None  # remove categories
pivot2 = pivot.reset_index()  # index to columns
pivot2 = pivot2.sort_values(by='overview_words_count', ascending=False)
display(pivot2.head(1))

,production_companies,overview_words_count
1041,Midnight Picture Show,175.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [155]:
answers['26']='1. Inside Out, The Dark Knight, 12 Years a Slave' # +

In [156]:
# Решение:
border = data.quantile(0.99, numeric_only=True)['vote_average']
one_pers_quantile_data = data.loc[data.vote_average > border]
display(one_pers_quantile_data)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,release_season,original_title_len,overview_words_count
9,tt2096673,175000000,853708609,Inside Out,Amy Poehler|Phyllis Smith|Richard Kind|Bill Ha...,Pete Docter,Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,Comedy|Animation|Family,Walt Disney Pictures|Pixar Animation Studios|W...,6/9/2015,8.0,2015,678708609,6,summer,10,112
34,tt3170832,6000000,35401758,Room,Brie Larson|Jacob Tremblay|Joan Allen|Sean Bri...,Lenny Abrahamson,Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,Drama|Thriller,Element Pictures|No Trace Camping|A24|Duperele...,10/16/2015,8.0,2015,29401758,10,fall,4,49
118,tt0816692,165000000,621752480,Interstellar,Matthew McConaughey|Jessica Chastain|Anne Hath...,Christopher Nolan,Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,Adventure|Drama|Science Fiction,Paramount Pictures|Legendary Pictures|Warner B...,11/5/2014,8.0,2014,456752480,11,fall,12,35
119,tt2015381,170000000,773312399,Guardians of the Galaxy,Chris Pratt|Zoe Saldana|Dave Bautista|Vin Dies...,James Gunn,All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,Action|Science Fiction|Adventure,Marvel Studios|Moving Picture Company (MPC)|Bu...,7/30/2014,7.9,2014,603312399,7,summer,23,28
125,tt2084970,14000000,233555708,The Imitation Game,Benedict Cumberbatch|Keira Knightley|Matthew G...,Morten Tyldum,The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,History|Drama|Thriller|War,Black Bear Pictures|Bristol Automotive,11/14/2014,8.0,2014,219555708,11,fall,18,46
128,tt2267998,61000000,369330363,Gone Girl,Ben Affleck|Rosamund Pike|Carrie Coon|Neil Pat...,David Fincher,You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,Mystery|Thriller|Drama,Twentieth Century Fox Film Corporation|Regency...,10/1/2014,7.9,2014,308330363,10,fall,9,30
138,tt2278388,30000000,174600318,The Grand Budapest Hotel,Ralph Fiennes|Tony Revolori|F. Murray Abraham|...,Wes Anderson,A perfect holiday without leaving home.,The Grand Budapest Hotel tells of a legendary ...,99,Comedy|Drama,Fox Searchlight Pictures|Scott Rudin Productio...,2/26/2014,7.9,2014,144600318,2,winter,24,66
370,tt1375666,160000000,825500000,Inception,Leonardo DiCaprio|Joseph Gordon-Levitt|Ellen P...,Christopher Nolan,Your mind is the scene of the crime.,"Cobb, a skilled thief who commits corporate es...",148,Action|Thriller|Science Fiction|Mystery|Adventure,Legendary Pictures|Warner Bros.|Syncopy,7/14/2010,7.9,2010,665500000,7,summer,9,44
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825,7,summer,15,67
872,tt0253474,35000000,120072577,The Pianist,Adrien Brody|Thomas Kretschmann|Frank Finlay|M...,Roman Polanski,Music was his passion. Survival was his master...,The Pianist is a film adapted from the biograp...,150,Drama|War,Bac Films|Canal+Polska|Heritage Films|Studio B...,9/24/2002,7.9,2002,85072577,9,fall,11,39


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [157]:
answers['27']='5. Daniel Radcliffe & Rupert Grint' # +

In [158]:
# Решение:
pairs = Counter()

for i in range(0, len(data)): # взято кол-во строк в массиве
    actors = data.cast[i].split('|')
    for j in list(combinations(actors,2)):
        if j not in pairs:
            pairs[j] = 1
        else:
            pairs[j] += 1

display(pairs.most_common(1))

[(('Daniel Radcliffe', 'Rupert Grint'), 8)]

ВАРИАНТ 2

# Submission

In [159]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [160]:
# и убедиться что ни чего не пропустил)
len(answers)

27